In [2]:
import os

if not os.path.exists('recognize-anything'):
    !git clone https://github.com/xinyu1205/recognize-anything.git
    
!pip install timm transformers fairscale pycocoevalcap
%cd recognize-anything

/home/ttd/hcmai/aic2024/multimedia-retrieval/training/tag/recognize-anything


/home/ttd/hcmai/aic2024/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import glob
import json
import torch
import numpy as np
from PIL import Image
from ram.models import ram_plus, ram
from ram import get_transform
from tqdm import tqdm

/home/ttd/hcmai/aic2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
def download_checkpoints(model, weight_path):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM_PLUS":
        !wget https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth -O pretrained/ram_plus_swin_large_14m.pth
    elif model == "RAM":
        !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
    else:
        print('Model not found')

weight_path = 'pretrained/ram_plus_swin_large_14m.pth'
weight_path1 = 'pretrained/ram_swin_large_14m.pth'
if not os.path.exists(weight_path):
    download_checkpoints("RAM_PLUS", weight_path)
else:
    print("RAM_PLUS weights already downloaded!")
    
    
if not os.path.exists(weight_path1):
    download_checkpoints("RAM", weight_path)
else:
    print("RAM weights already downloaded!")

RAM_PLUS weights already downloaded!
You selected RAM
--2024-07-27 15:38:12--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 54.230.71.28, 54.230.71.103, 54.230.71.2, ...
Connecting to huggingface.co (huggingface.co)|54.230.71.28|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-07-27 15:38:13--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3

# Parse data path

In [5]:
keyframes_dir = '../../Keyframes'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_keyframe_paths[data_part] =  dict()
    

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'
    video_dirs = sorted(os.listdir(data_part_path))
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.webp'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

# Train with GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
transform = get_transform(image_size=384)
model = ram_plus(pretrained='pretrained/ram_plus_swin_large_14m.pth',
            image_size=384,
            vit='swin_l')
model.eval()
model = model.to(device)

model1 = ram(pretrained='pretrained/ram_swin_large_14m.pth',
            image_size=384,
            vit='swin_l')
model1.eval()
model1 = model1.to(device)

--------------
pretrained/ram_plus_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_plus_swin_large_14m.pth
vit: swin_l


In [ ]:
!rm -r /kaggle/working/recognize-anything

In [19]:
import torch.nn.functional as F


@torch.no_grad()
def generate_tag(model,
                image
                ):

    image_embeds = model.image_proj(model.visual_encoder(image))
    image_atts = torch.ones(image_embeds.size()[:-1],
                            dtype=torch.long).to(image.device)

    image_cls_embeds = image_embeds[:, 0, :]
    image_spatial_embeds = image_embeds[:, 1:, :]

    bs = image_spatial_embeds.shape[0]

    des_per_class = int(model.label_embed.shape[0] / model.num_class)

    image_cls_embeds = image_cls_embeds / image_cls_embeds.norm(dim=-1, keepdim=True)
    reweight_scale = model.reweight_scale.exp()
    logits_per_image = (reweight_scale * image_cls_embeds @ model.label_embed.t())
    logits_per_image = logits_per_image.view(bs, -1,des_per_class)

    weight_normalized = F.softmax(logits_per_image, dim=2)
    label_embed_reweight = torch.empty(bs, model.num_class, 512).to(image.device).to(image.dtype)

    for i in range(bs):
        reshaped_value = model.label_embed.view(-1, des_per_class, 512)
        product = weight_normalized[i].unsqueeze(-1) * reshaped_value
        label_embed_reweight[i] = product.sum(dim=1)

    label_embed = torch.nn.functional.relu(model.wordvec_proj(label_embed_reweight))

    # recognized image tags using alignment decoder
    tagging_embed = model.tagging_head(
        encoder_embeds=label_embed,
        encoder_hidden_states=image_embeds,
        encoder_attention_mask=image_atts,
        return_dict=False,
        mode='tagging',
    )

    logits = model.fc(tagging_embed[0]).squeeze(-1)

    targets = torch.where(
        torch.sigmoid(logits) > model.class_threshold.to(image.device),
        torch.tensor(1.0).to(image.device),
        torch.zeros(model.num_class).to(image.device))

    tag = targets.cpu().numpy()
    tag[:,model.delete_tag_index] = 0
    tag_output = []
    tag_output_chinese = []
    for b in range(bs):
        index = np.argwhere(tag[b] == 1)
        token = model.tag_list[index].squeeze(axis=1)
        tag_output.append(' | '.join(token))
        token_chinese = model.tag_list_chinese[index].squeeze(axis=1)
        tag_output_chinese.append(' | '.join(token_chinese))


    return tag_output, 

image_path = "/home/ttd/hcmai/aic2024/multimedia-retrieval/training/Keyframes/L02/V001/0002.webp"


generate_tag(model1, transform(Image.open(image_path)).unsqueeze(0).to(device))
generate_tag(model, transform(Image.open(image_path)).unsqueeze(0).to(device))

(['chair | person | interview | man | news | stand | stool | stop sign | television | video | woman'],)